In [ ]:
# Package Author Mark Benmuvhar
# pdf_modify_prod_csv
# Version 2.0.0
# 10/16/2022

In [ ]:
import numpy as np
import pandas as pd
import re
import os

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html?highlight=contains
def get_levo_bool(df):
    try:
        return df['ds_name'].str.contains(r'LEVOTHYROXINE SODIUM\s*\*\*')
    except:
        print('error getting name from ', s,'/n Found ', m)

In [ ]:
def get_levo_be(df):
    try:
        return df['be'].str.contains(r'--\>*')
    except:
        print('error getting name from ', s,'/n Found ', m)

In [ ]:
# find_be takes a cell from the  strength column from Levothyroxine Sodium
# and abstracts the BE component 
# It applies a regex to abstract
# Intended for use in an apply statement

def find_be(s):
    try:
        #p=re.compile('\s(\D\D\d,*)+')
        p=re.compile('(AB\d,*)+')
        m=p.search(s)
        return m[0]
    except:
        print('error getting name from ', s,'/n Found ', m)

In [ ]:
# find_strength takes a cell from the strength column from Levothyroxine Sodium
# and abstracts the strength component 
# It applies a regex to abstract
# Intended for use in an apply statement
def find_strength(s):  
    try:
        p=re.compile('0\.\d*MG')
        m=p.search(s)
        return m[0]
    except:
        print('error getting strength from ', s)

In [ ]:
def find_app_type(s):
    try:
        if pd.notna(s):
            p=re.compile(r'A|N')
            m=p.search(s)
            return m[0]
        else:
            return np.nan
    except:
        print('error finding application type in ', s)

In [ ]:
def find_app_nums(s):
    try:
        if pd.notna(s):
            p=re.compile(r'\d+')
            m=p.search(s)
            return m[0]
        else:
            return np.nan
    except:
        print('error finding application numbers in ', s)

In [ ]:
s=['bob;jane','jack; diane']
p = re.compile(r';(?!\s)')
for i in s:
    print(i)
    print(p.search(i))

In [ ]:
#def main():
  
directory = ('OrangeBook\csv\prods\\')
otpt_directory = ('OrangeBook\csv_edits\prods\\')
otpt_directory2 = ('OrangeData\prods\\')

p = re.compile(r'prod_v1_4')
p_sem = re.compile(r';(?!\s)')

# file naming variables
for filename in os.scandir(directory):
    if not filename.is_dir():
        try:
            print(filename)
            clean=pd.read_csv(filename.path)
            df=clean.copy()
        except Exception as e:
            print('File Input error in ',filename, '\n',e)

# Editing Block 
    #Cleanup of Levothyroxine labels
        df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True )
    
        idx=get_levo_bool(df)
        idx2=get_levo_be(df)
        idx3=idx&idx2
        
        df.loc[idx, 'ds_name'] = 'LEVOTHYROXINE SODIUM'
        
        be=df.loc[idx3, 'strength'].apply(find_be)
        strengths = df.loc[idx3, 'strength'].apply(find_strength)
        df.loc[idx3, 'be'] = be
        df.loc[idx3, 'strength'] = strengths
        
        app_type = df['app_no'].apply(find_app_type)
        app_nums = df['app_no'].apply(find_app_nums)
        df['app_type']= app_type
        df['app_nums']= app_nums
        
    #Cleanup of other columns
        df['prod_no'].replace(np.nan, None, inplace = True)
        df['be'].replace(np.nan, None, inplace = True)
        df['note'].replace(np.nan, 'No Notes', inplace = True)
        df['prod_no'] = df['prod_no'].transform(str)
        df['prod_no'] = df['prod_no'].str.replace(r'00+\s*','', regex = True)
        df['route'].replace(p_sem,'; ', inplace = True)
        df['ds_name']=df['ds_name'].str.lstrip()
        df = df.drop(['app_no'], axis=1)

#file Output
        rn_out_name = p.sub('prod_v2_0_ed1.csv', filename.name)
        rn_out_doc = otpt_directory+rn_out_name
        df.to_csv(rn_out_doc)
        rn_out_doc = otpt_directory2+rn_out_name
        df.to_csv(rn_out_doc)
        print('Completed cleaning drug product info from ',filename.name)
        print('File written to ',rn_out_doc)